# Semantic Segmentation Exercise POSTER 2

The data used for this exercise stems from /dtu/datasets1/0251
The data loader below assumes that you are working on the HPC machines with the data located at  

/dtu/datasets1/02516/phc_data

We first load some libraries

In [ ]:
import os
import numpy as np
import glob
import PIL.Image as Image

# pip install torchsummary
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import models
from torchsummary import summary
import torch.optim as optim
from time import time

import matplotlib.pyplot as plt
from IPython.display import clear_output

Next, the data loader

In [ ]:
data_path = '/dtu/datasets1/02516/phc_data'
#data_path = './phc_data'
class DataEyeLoader(torch.utils.data.Dataset):
    def __init__(self, train, transform, data_path=data_path):
        'Initialization'
        self.transform = transform
        data_path = os.path.join(data_path, 'train' if train else 'test')
        self.image_paths = sorted(glob.glob(data_path + '/images/*.jpg'))
        self.label_paths = sorted(glob.glob(data_path + '/labels/*.png'))
        
    def __len__(self):
        'Returns the total number of samples'
        return len(self.image_paths)

    def __getitem__(self, idx):
        'Generates one sample of data'
        image_path = self.image_paths[idx]
        label_path = self.label_paths[idx]
        
        image = Image.open(image_path)
        label = Image.open(label_path)
        Y = self.transform(label)
        X = self.transform(image)
        return X, Y

In [ ]:
data_path = '/dtu/datasets1/02516/phc_data'
#data_path = './phc_data'
class DataSkinLoader(torch.utils.data.Dataset):
    def __init__(self, train, transform, data_path=data_path):
        'Initialization'
        self.transform = transform
        data_path = os.path.join(data_path, 'train' if train else 'test')
        self.image_paths = sorted(glob.glob(data_path + '/images/*.jpg'))
        self.label_paths = sorted(glob.glob(data_path + '/labels/*.png'))
        
    def __len__(self):
        'Returns the total number of samples'
        return len(self.image_paths)

    def __getitem__(self, idx):
        'Generates one sample of data'
        image_path = self.image_paths[idx]
        label_path = self.label_paths[idx]
        
        image = Image.open(image_path)
        label = Image.open(label_path)
        Y = self.transform(label)
        X = self.tran

In principle, the images could have different sizes. Let's resize them all to $128\times128$ pixels, using the torchvision Resize. 

size = 128
train_transform = transforms.Compose([transforms.Resize((size, size)), 
                                    transforms.ToTensor()])
test_transform = transforms.Compose([transforms.Resize((size, size)), 
                                    transforms.ToTensor()])

batch_size = 6
trainset = DataEyeLoader(train=True, transform=train_transform)
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=3)
testset = PhC(train=False, transform=test_transform)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=3)

In [ ]:
print('Loaded %d training images' % len(trainset))
print('Loaded %d test images' % len(testset))

Let's look at some images from the dataset!

In [ ]:
plt.rcParams['figure.figsize'] = [18, 6]

images, labels = next(iter(train_loader))

for i in range(6):
    plt.subplot(2, 6, i+1)
    plt.imshow(np.swapaxes(np.swapaxes(images[i], 0, 2), 0, 1))

    plt.subplot(2, 6, i+7)
    plt.imshow(labels[i].squeeze())
plt.show()

## Device

Check if GPU is available.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

First, we consider a simple encoder decoder network for image registration

In [ ]:
class EncDec(nn.Module):
    def __init__(self):
        super().__init__()

        # encoder (downsampling)
        self.enc_conv0 = nn.Conv2d(3, 64, 3, padding=1)
        self.pool0 = nn.MaxPool2d(2, 2)  # 128 -> 64
        self.enc_conv1 = nn.Conv2d(64, 64, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)  # 64 -> 32
        self.enc_conv2 = nn.Conv2d(64, 64, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)  # 32 -> 16
        self.enc_conv3 = nn.Conv2d(64, 64, 3, padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)  # 16 -> 8

        # bottleneck
        self.bottleneck_conv = nn.Conv2d(64, 64, 3, padding=1)

        # decoder (upsampling)
        self.upsample0 = nn.Upsample(16)  # 8 -> 16
        self.dec_conv0 = nn.Conv2d(64, 64, 3, padding=1)
        self.upsample1 = nn.Upsample(32)  # 16 -> 32
        self.dec_conv1 = nn.Conv2d(64, 64, 3, padding=1)
        self.upsample2 = nn.Upsample(64)  # 32 -> 64
        self.dec_conv2 = nn.Conv2d(64, 64, 3, padding=1)
        self.upsample3 = nn.Upsample(128)  # 64 -> 128
        self.dec_conv3 = nn.Conv2d(64, 1, 3, padding=1)

    def forward(self, x):
        # encoder
        e0 = self.pool0(F.relu(self.enc_conv0(x)))
        e1 = self.pool1(F.relu(self.enc_conv1(e0)))
        e2 = self.pool2(F.relu(self.enc_conv2(e1)))
        e3 = self.pool3(F.relu(self.enc_conv3(e2)))

        # bottleneck
        b = F.relu(self.bottleneck_conv(e3))

        # decoder
        d0 = F.relu(self.dec_conv0(self.upsample0(b)))
        d1 = F.relu(self.dec_conv1(self.upsample1(d0)))
        d2 = F.relu(self.dec_conv2(self.upsample2(d1)))
        d3 = self.dec_conv3(self.upsample3(d2))  # no activation
        return d3

We implement the following metrics for validating segmentation performance:  Dice overlap, Intersection overUnion, Accuracy, Sensitivity, and Specificy.

In [ ]:
# Dice overalp

def dice_coefficient(y_true, y_pred):
    smooth = 1e-6  # To avoid division by zero
    y_true_f = y_true.view(-1)
    y_pred_f = y_pred.view(-1)
    intersection = (y_true_f * y_pred_f).sum()
    return (2. * intersection + smooth) / (y_true_f.sum() + y_pred_f.sum() + smooth)


def dice_coefficient(y_pred, y_true, epsilon=1e-07):
    y_pred_copy = prediction.clone()

    y_pred_copy[prediction_copy < 0] = 0
    y_pred_copy[prediction_copy > 0] = 1

    intersection = abs(torch.sum(y_pred_copy * y_true))
    union = abs(torch.sum(y_pred_copy) + torch.sum(y_true))
    dice = (2. * intersection + epsilon) / (union + epsilon)
    return dice

# Intersection overUnion

def intersection_over_union(y_true, y_pred):
    smooth = 1e-6  # To avoid division by zero
    y_true_f = y_true.view(-1)
    y_pred_f = y_pred.view(-1)
    intersection = (y_true_f * y_pred_f).sum()
    union = y_true_f.sum() + y_pred_f.sum() - intersection
    return intersection / (union + smooth)

# Accuracy

def accuracy(y_true, y_pred):
    y_pred = y_pred.round()  # Convert probabilities to binary
    correct = (y_true == y_pred).float()  # Check if predictions are correct
    return correct.sum() / correct.numel()

# Sensitivity (measures true positives)

def sensitivity(y_true, y_pred):
    y_pred = y_pred.round()  # Convert probabilities to binary
    true_positives = (y_true * y_pred).sum()
    possible_positives = y_true.sum()
    return true_positives / (possible_positives + 1e-6)  # Avoid division by zero

# Specificity(measures true negatives)

def specificity(y_true, y_pred):
    y_pred = y_pred.round()  # Convert probabilities to binary
    true_negatives = ((1 - y_true) * (1 - y_pred)).sum()
    possible_negatives = (1 - y_true).sum()
    return true_negatives / (possible_negatives + 1e-6)  # Avoid division by zero



In [ ]:
def calculate_segmentation_metrics(y_true, y_pred, threshold=0.5):
    y_pred = y_pred.round()
    TP = ((y_true == 1) & (y_pred == 1)).sum().item()
    FP = ((y_true == 0) & (y_pred == 1)).sum().item()
    TN = ((y_true == 0) & (y_pred == 0)).sum().item()
    FN = ((y_true == 1) & (y_pred == 0)).sum().item()
    # Calcular Dice
    dice = (2 * TP) / (2 * TP + FP + FN) if (2 * TP + FP + FN) > 0 else 0

    # Calcular IoU
    iou = TP / (TP + FP + FN) if (TP + FP + FN) > 0 else 0

    # Calcular Accuracy
    accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0

    # Calcular Sensibilidad
    sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0

    # Calcular Especificidad
    specificity = TN / (TN + FP) if (TN + FP) > 0 else 0


    metrics = {
        'Dice': dice,
        'IoU': iou,
        'Accuracy': accuracy,
        'Sensitivity': sensitivity,
        'Specificity': specificity
    }
    return metrics

In [ ]:
import torch
from torchmetrics.classification import BinaryAccuracy, BinaryJaccardIndex, BinaryRecall, BinarySpecificity, BinaryDiceCoefficient

def calculate_segmentation_metrics(y_true,y_pred):
y_pred = y_pred.round()

dice_score2=Dice(y_pred, y_true)

dice_score  = BinaryDiceCoefficient(y_pred,y_true)
iou_score = BinaryJaccardIndex(y_pred,y_true)
accuracy_score = BinaryAccuracy(y_pred,y_true)
sensitivity_score=BinaryRecall(y_pred,y_true)
specificity_score=BinarySpecificity(y_pred,y_true)

metrics = {
        'Dice': dice,
        'IoU': iou,
        'Accuracy': accuracy,
        'Sensitivity': sensitivity,
        'Specificity': specificity
    }
return metrics


Implement  a  U-net  architecture  to  segment  the  image. We will use Convolutions with stride 2 for downsampling and ConvTranspose for upsampling.

In [ ]:
class UNet2:
    def __init__(self):
        super().__init__()

        # Encoder
        self.enc1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.enc2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.enc3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        
        # Bottleneck
        self.bottleneck = nn.Conv2d(256, 512, kernel_size=3, padding=1)

        # Decoder
        self.dec1 = nn.Conv2d(512, 256, kernel_size=3, padding=1)
        self.dec2 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.dec3 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.out_conv = nn.Conv2d(64, 1, kernel_size=1)

        # Convoluciones para downsampling y upsampling
        self.downsample1 = nn.Conv2d(64, 64, kernel_size=3, padding=1, stride=2)  # Reemplazado
        self.downsample2 = nn.Conv2d(128, 128, kernel_size=3, padding=1, stride=2)  # Reemplazado
        self.downsample3 = nn.Conv2d(256, 256, kernel_size=3, padding=1, stride=2)  # Reemplazado
        self.upsample1 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)  # Usando transposed convolutions
        self.upsample2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)  # Usando transposed convolutions
        self.upsample3 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)  # Usando transposed convolutions

    def forward(self, x):
        # Encoder
        e1 = F.relu(self.enc1(x))
        e2 = F.relu(self.downsample1(e1))  # Usamos convolución con stride=2
        e2 = F.relu(self.enc2(e2))
        e3 = F.relu(self.downsample2(e2))  # Usamos convolución con stride=2
        e3 = F.relu(self.enc3(e3))
        b = F.relu(self.bottleneck(e3))

        # Decoder
        d1 = self.upsample1(b)
        d1 = torch.cat((d1, e3), dim=1)  # Skip connection
        d1 = F.relu(self.dec1(d1))
        d2 = self.upsample2(d1)
        d2 = torch.cat((d2, e2), dim=1)  # Skip connection
        d2 = F.relu(self.dec2(d2))
        d3 = self.upsample3(d2)
        d3 = torch.cat((d3, e1), dim=1)  # Skip connection
        d3 = F.relu(self.dec3(d3))

        return self.out_conv(d3)

Different Loss functions

In [ ]:
def bce_loss(y_real, y_pred):
    return torch.mean(y_pred - y_real*y_pred + torch.log(1 + torch.exp(-y_pred)))

In [ ]:
def dice_loss(y_real, y_pred):
    return 1 - ((2*y_real*y_pred + 1).mean())/((y_real+y_pred).mean()+1)

Run it

In [ ]:
model = EncDec().to(device)
train(model, optim.Adam(model.parameters(), 0.0001), dice_loss, 20, train_loader, test_loader)

Focal Loss

In [ ]:
gamma = 2
def focal_loss(y_real, y_pred,gamma):
    y_pred_sig = torch.sigmoid(y_pred)
    term = (1-y_pred_sig)**gamma * y_real * torch.log(y_pred_sig) + (1-y_real)torch.log(1-y_pred_sig)
    return (-term.sum())

In [ ]:
model = EncDec().to(device)
train(model, optim.Adam(model.parameters(), 0.0001), focal_loss, 20, train_loader, test_loader)